In [ ]:
import cv2
import numpy as np
import pytesseract
import folium

# Set up the path to the Tesseract executable (on Windows)
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# Load YOLO model (Make sure to use a model trained on traffic signs)

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # YOLO model files (weights and config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Traffic sign class labels (Replace with the specific labels for your model if available)
class_labels = ["stop sign", "yield sign", "speed limit", "pedestrian crossing", "no entry", "traffic light"]

# List to store GPS coordinates and corresponding traffic sign details
gps_log = []

# Simulate a GPS sensor reading every frame (In a real application, you'd get live data)
def get_gps_data():
    # This should be replaced with actual GPS reading code
    gps_data = {
        "latitude": 37.7749 + np.random.uniform(-0.0005, 0.0005),  # Simulate slight changes in location
        "longitude": -122.4194 + np.random.uniform(-0.0005, 0.0005)
    }
    return gps_data
    
# Open video capture (webcam)
cap = cv2.VideoCapture(0)
while True:
    # Read a frame from the camera
    ret, frame = cap.read()
    if not ret:
        break
    h, w = frame.shape[:2]
    
    # Preprocess the image for YOLO (Resize to 416x416)
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Set the input to the YOLO model
    net.setInput(blob)

    # Perform forward pass to get detections
    detections = net.forward(output_layers)

    # Loop through each detection
    class_ids = []
    confidences = []
    boxes = []

    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Threshold for confidence
                # Get bounding box coordinates (x, y, w, h)
                center_x = int(obj[0] * w)
                center_y = int(obj[1] * h)
                box_w = int(obj[2] * w)
                box_h = int(obj[3] * h)
            
                # Rectangle coordinates
                x = int(center_x - box_w / 2)
                y = int(center_y - box_h / 2)

                boxes.append([x, y, box_w, box_h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression (NMS) to remove duplicate boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Get the current GPS coordinates (simulated here)
    gps_data = get_gps_data()

    # Process each detection and log the GPS coordinates
    for i in range(len(boxes)):
        if i in indexes:
            x, y, box_w, box_h = boxes[i]
            label = class_labels[class_ids[i]]
            color = (0, 255, 0)  # Green for bounding box
            cv2.rectangle(frame, (x, y), (x + box_w, y + box_h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
            # Log the GPS coordinates along with the detection label
            gps_log.append({
                "latitude": gps_data["latitude"],
                "longitude": gps_data["longitude"],
                "label": label
            })

    # Display the frame with detected traffic signs
    cv2.imshow("Traffic Sign Detection with GPS", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the window
cap.release()
cv2.destroyAllWindows()

# After the drive, plot the GPS locations on a map
# Create a folium map centered around the first GPS coordinate
m = folium.Map(location=[gps_log[0]["latitude"], gps_log[0]["longitude"]], zoom_start=15)

# Add markers to the map for each detected sign
for entry in gps_log:
    folium.Marker(
        location=[entry["latitude"], entry["longitude"]],
        popup=f"Traffic Sign: {entry['label']}",
        icon=folium.Icon(color="red")
    ).add_to(m)
    
# Save the map to an HTML file
m.save("traffic_signs_map.html")